In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mlflow
import numpy as np
import datetime

from lib.reproduction import major_oxides
from lib.full_flow_dataloader import load_full_flow_data
from lib.norms import Norm1Scaler, Norm3Scaler
from sklearn.ensemble import GradientBoostingRegressor
from lib.get_preprocess_fn import get_preprocess_fn
from lib.cross_validation import CustomKFoldCrossValidator, get_cross_validation_metrics, perform_cross_validation
from lib.metrics import rmse_metric, std_dev_metric

train_processed, test_processed = load_full_flow_data()

In [3]:
drop_cols = major_oxides + ["ID", "Sample Name"]
norm = 3

In [4]:
models = []
rmse_scores = []

gbr_params = {
    'n_estimators': 100,
    'max_depth': 3,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': None,
    'loss': 'squared_error',
    'learning_rate': 0.1,
    'subsample': 1.0,
    'criterion': 'friedman_mse',
    'random_state': 42,
    'verbose': 0,
    'validation_fraction': 0.1,
    'n_iter_no_change': None,
    'tol': 1e-4,
    'ccp_alpha': 0.0
}

mlflow.set_experiment(f'GBR_Norm{norm}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')

for target in major_oxides:
    with mlflow.start_run(run_name=f"GBR_{target}"):
        # == Cross Validation ==
        kf = CustomKFoldCrossValidator(k=5, random_state=42, group_by="Sample Name")
        scaler = Norm1Scaler() if norm == 1 else Norm3Scaler()

        cv_metrics = perform_cross_validation(
            model=GradientBoostingRegressor(**gbr_params),
            preprocess_fn=get_preprocess_fn(target_col=target, drop_cols=drop_cols, preprocessor=scaler),
            kf=kf,
            data=train_processed,
            metric_fns=[rmse_metric, std_dev_metric],
        )

        mlflow.log_metrics(get_cross_validation_metrics(cv_metrics).as_dict())

        # == Training ==
        preprocess_fn = get_preprocess_fn(target_col=target, drop_cols=drop_cols, preprocessor=scaler)
        X_train, y_train, X_test, y_test = preprocess_fn(train_processed, test_processed)

        # Train the model
        model = GradientBoostingRegressor(**gbr_params)
        model.fit(X_train, y_train)
        models.append(model)

        pred = model.predict(X_test)
        rmse = rmse_metric(y_test, pred)
        std_dev = std_dev_metric(y_test, pred)

        # Logging
        mlflow.log_params({
            **gbr_params,
            "target": target,
            "norm": norm
        })
        mlflow.log_metrics({"rmse": rmse, "std_dev": std_dev})
        mlflow.sklearn.log_model(model, f"model_{target}")

2024/05/06 12:45:33 INFO mlflow.tracking.fluent: Experiment with name 'GBR_Norm3_20240506-124533' does not exist. Creating a new experiment.
